In [2]:
%gui qt5
%matplotlib qt5
from functools import partial as _partial
import numpy as np
from scipy.io import loadmat
import scipy.signal as scy_sig

import matplotlib.pyplot as plt
import matplotlib.gridspec as mpl_gs
import matplotlib.cm as cmap
from matplotlib import rcParams
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection, LineCollection
from matplotlib import colors as mcolors
rcParams.update(
    {'font.size': 16,'lines.linewidth': 2, 'axes.grid': True})

from siriuspy.devices import BunchbyBunch
from mathphys.functions import load_pickle

In [21]:
def process_data(data, center_freq, sigma_freq, T_rev, downsample=1):
    """."""
    # remove DC component from bunches
    data = data - data.mean(axis=0)[None, :]
    
    # get the analytic data vector, via discrete hilbert transform
    data_anal = scy_sig.hilbert(data, axis=0)

    # calculate DFT:
    data_dft = np.fft.fft(data_anal, axis=0)
    freq = np.fft.fftfreq(data_anal.shape[0], d=T_rev*downsample)
    
    # Apply Gaussian filter to get only the synchrotron frequency
    H = np.exp(-(freq - center_freq)**2/2/sigma_freq**2)
    H += np.exp(-(freq + center_freq)**2/2/sigma_freq**2)
    H /= H.max()
    data_dft *= H[:, None]

    # compensate the different time samplings of each bunch:
    dts = np.arange(data_anal.shape[1])/data_anal.shape[1] * T_rev
    comp = np.exp(-1j*2*np.pi * freq[:, None]*dts[None, :])
    data_dft *= comp
    
    # get the processed data by inverse DFT
    data_anal = np.fft.ifft(data_dft, axis=0)
    
    # decompose data into even fill eigenvectors:
    data_modes = np.fft.fft(data_anal, axis=1) / data_anal.shape[1]
    return data_anal, data_dft/data_anal.shape[0], data_modes

def waterfall_plot(axis, xs, zs, data):
    vertcs, colors = [], []
    cors = ['b', 'r', 'g', 'y', 'm', 'c']
    for i, y in enumerate(zs):
        ys = data[:, i].copy()
        ys[0], ys[-1] = 0, 0
        vertcs.append(list(zip(xs, ys)))
        colors.append(cors[i%len(cors)])
    poly = PolyCollection(
        vertcs, closed=False, edgecolors='k',
        linewidths=1, facecolors=colors)
    
    poly.set_alpha(0.7)
    axis.add_collection3d(poly, zs=zs, zdir='x')

def fit_exponential(times, data, t_ini=None, t_fin=None):
    t_ini = t_ini or times.min()
    t_fin = t_fin or times.max()
    idx = (times >= t_ini) & (times <= t_fin)
    tim = times[idx]
    dt = data[idx]
    dt = np.log(dt)
    coef = np.polynomial.polynomial.polyfit(tim, dt, deg=1, rcond=None)
    return tim, coef

# Load data from matlab

In [77]:
a = loadmat('prm.mat')

In [78]:
# idx = -2  # data
# idx = 17  # proc_ds
idx = 16  # ds
a['prm'][0].dtype.descr[idx], a['prm'][0][0][idx]

(('ds', '|O'), array([[1]], dtype=uint8))

In [79]:
data = a['prm'][0][0][-2].reshape((-1, 148))
data

array([[-34,   5,  13, ...,  41,  -3,  20],
       [-33,   6,  11, ...,  37,  -1,  18],
       [-35,   6,  10, ...,  39,  -2,  17],
       ...,
       [-58, -21, -15, ...,   7, -28, -11],
       [-60, -25, -18, ...,  11, -31,  -9],
       [-63, -26, -15, ...,   0,   0,   0]], dtype=int16)

In [80]:
frf = 476000000
Trf = 1/frf
harm_num = 148
T0 = harm_num * Trf
f0 = 1/T0
downsample = a['prm'][0][0][idx][0][0]

# Read data PVs

In [3]:
bbb = BunchbyBunch('L')

In [9]:
frf = bbb.info.rf_freq_nom * 1e6
Trf = 1/frf
harm_num = bbb.info.harmonic_number
T0 = harm_num * Trf
f0 = 1/T0
downsample = bbb.sram.downsample
data = bbb.sram.data_raw

TypeError: 'NoneType' object is not subscriptable

# Read saved data

In [7]:
%ls -lh data*

-rw-r--r-- 1 fernando facs 433K out  2 12:45 data_001.pickle
-rw-r--r-- 1 fernando facs 433K out  2 12:45 data_002.pickle
-rw-r--r-- 1 fernando facs 433K out  2 12:45 data_003.pickle
-rw-r--r-- 1 fernando facs 757K out  2 12:45 data_004.pickle
-rw-r--r-- 1 fernando facs 865K out  2 12:45 data_005.pickle
-rw-r--r-- 1 fernando facs 1.2M out  2 12:45 data_006.pickle
-rw-r--r-- 1 fernando facs 1.2M out  2 12:45 data_007.pickle
-rw-r--r-- 1 fernando facs 595K out  2 14:13 data_008.pickle
-rw-r--r-- 1 fernando facs  12M out  2 14:20 data_009.pickle


In [22]:
data_dict = load_pickle('data_009.pickle')

In [23]:
frf = data_dict['frf']
Trf = 1/frf
harm_num = data_dict['harm_num']
T0 = harm_num * Trf
f0 = 1/T0
downsample = data_dict['downsample']
data = data_dict['data'].reshape((-1, harm_num))

# Process Data

In [24]:
data2 = data - data.mean(axis=1)[:, None]
data2 -= data2.mean(axis=0)[None, :]
data2_dft = np.fft.rfft(data2, axis=0)
rfreq = np.fft.rfftfreq(data2.shape[0], d=T0*downsample)

avg_data2 = np.abs(data2_dft).mean(axis=1)

f  = plt.figure(figsize=(7, 4))
gs = mpl_gs.GridSpec(1, 1)
gs.update(left=0.15, right=0.95, top=0.97, bottom=0.18, wspace=0.35, hspace=0.2)
aty = plt.subplot(gs[0, 0])

aty.plot(rfreq[1:], avg_data2[1:])
aty.set_yscale('log')
f.show()

In [33]:
data_anal, data_dft, data_modes = process_data(
    data, center_freq=2100, sigma_freq=200,
    T_rev=T0, downsample=downsample)

In [34]:
f  = plt.figure(figsize=(12, 9))
gs = mpl_gs.GridSpec(2, 2)
gs.update(left=0.10, right=0.95, top=0.97, bottom=0.10, wspace=0.35, hspace=0.2)
# aty = plt.subplot(gs[0, :7], projection='3d')
# afy = plt.subplot(gs[1, :7], projection='3d')
aty = plt.subplot(gs[0, 0])
afy = plt.subplot(gs[1, 0])
atx = plt.subplot(gs[0, 1])
afx = plt.subplot(gs[1, 1])

t = np.arange(data_anal.shape[0]) * T0 * downsample * 1e3
modes = np.arange(data_anal.shape[1])
bunch_nums = modes + 1

abs_modes = np.abs(data_modes)
abs_dataf = np.abs(data_anal)

afx.plot(modes, abs_modes.std(axis=0))
afx.set_xlabel('mode number')
afx.set_ylabel('rms amplitude [a.u.]')

# waterfall_plot(afy, t, modes, abs_modes)
# afy.view_init(elev=35.5, azim=-135)
# afy.set_ylim3d(t.min(), t.max())
# afy.set_xlim3d(modes.min(), modes.max())
# afy.set_zlim3d(0, abs_modes.max())
# afy.set_ylabel('\ntime [ms]')
# afy.set_xlabel('\nmode number')
# afy.set_zlabel('amplitude')
M, T = np.meshgrid(modes, t)
cf = afy.pcolormesh(
    T, M, abs_modes, cmap='jet',
    vmin=abs_modes.min(), vmax=abs_modes.max())
afy.set_xlabel('Time [ms]')
afy.set_ylabel('Mode Number')
cb = f.colorbar(cf, ax=afy, pad=0.01)
cb.set_label('Amplitude [a.u.]')

atx.plot(bunch_nums, abs_dataf.std(axis=0))
atx.set_xlabel('Bunch Number')
atx.set_ylabel('RMS Amplitude [a.u.]')

# waterfall_plot(aty, t, bunch_nums, abs_dataf)
# aty.view_init(elev=35.5, azim=-135)
# aty.set_ylim3d(t.min(), t.max())
# aty.set_xlim3d(bunch_nums.min(), bunch_nums.max())
# aty.set_zlim3d(0, abs_dataf.max())
# aty.set_ylabel('\ntime [ms]')
# aty.set_xlabel('\nbunch number')
# aty.set_zlabel('amplitude')
M, T = np.meshgrid(bunch_nums, t)
cf = aty.pcolormesh(
    T, M, abs_dataf, cmap='jet',
    vmin=abs_dataf.min(), vmax=abs_dataf.max())
aty.set_xlabel('Time [ms]')
aty.set_ylabel('Bunch Number')
cb = f.colorbar(cf, ax=aty, pad=0.01)
cb.set_label('Amplitude [a.u.]')

f.show()

In [32]:
mode_num = 674
data_mode = data_modes[:, mode_num]
abs_mode = np.abs(data_mode)
timg, coefg = fit_exponential(t, abs_mode, t_ini=0.05, t_fin=0.70)
timd, coefd = fit_exponential(t, abs_mode, t_ini=1.15, t_fin=1.65)

f  = plt.figure(figsize=(7, 8))
gs = mpl_gs.GridSpec(2, 1)
gs.update(left=0.15, right=0.95, top=0.96, bottom=0.1, hspace=0.2)
aty = plt.subplot(gs[0, 0])
atx = plt.subplot(gs[1, 0])

fitg = np.exp(coefg[0] + timg*coefg[1])
fitd = np.exp(coefd[0] + timd*coefd[1])
szg = fitg.size
szd = fitd.size

aty.plot(t, abs_mode, label='Data')
aty.plot(timg, fitg, label='Grow Fit')
aty.plot(timd, fitd, label='Damp Fit')

aty.legend(loc='best', fontsize='small')

aty.annotate(
    f'rate = {coefg[1]:.2f} kHz', fontsize='x-small',
    xy=(timg[szg//2], fitg[szg//2]),
    textcoords='offset points', xytext=(-100, 10), 
    arrowprops=dict(arrowstyle='->'),
    bbox=dict(boxstyle="round", fc="0.8"))

aty.annotate(
    f'rate = {coefd[1]:.2f} kHz', fontsize='x-small',
    xy=(timd[szd//2], fitd[szd//2]),
    textcoords='offset points', xytext=(10, 10),
    arrowprops=dict(arrowstyle='->'),
    bbox=dict(boxstyle="round", fc="0.8"))

aty.set_title(f'Evolution of mode {mode_num:d}')
aty.set_xlabel('time [ms]')
aty.set_ylabel('Amplitude [a.u.]')

inst_freq = np.diff(np.unwrap(np.angle(data_mode)))/2/np.pi
atx.plot(t[:-1], inst_freq/T0/downsample/1e3)
atx.set_xlabel('time [ms]')
atx.set_ylabel('Instantaneous Frequency [kHz]')

f.show()

## Simulated Data

In [6]:
freq_tune = 0.005
growth_time = 5e-3
mode = 134
t = np.arange(data.size * downsample ) * Trf
T = t.reshape((-1, harm_num))[::downsample, :]
t = T.ravel()
y = np.exp((1/growth_time + 1j*2*np.pi*f0*(mode + freq_tune))*t).real
y += np.exp((0/1 + 1j*2*np.pi*f0*(0 + freq_tune))*t).real
Y = y.reshape((-1, harm_num))

1.6